## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hithadhoo,MV,2022-02-14 04:58:54,-0.6000,73.0833,82.36,72,79,12.97
1,1,Marzuq,YE,2022-02-14 04:58:55,14.4000,46.4667,59.04,41,34,3.42
2,2,Berezovyy,RU,2022-02-14 04:58:56,51.6667,135.7000,16.83,85,100,3.58
3,3,Hobart,AU,2022-02-14 04:55:23,-42.8794,147.3294,71.02,65,75,9.22
4,4,La Ronge,CA,2022-02-14 04:58:56,55.1001,-105.2842,-0.15,78,0,5.75


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?15
What is the maximum temperature you would like for your trip?30


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"]= ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Berezovyy,RU,16.83,51.6667,135.7000,
21,Bethel,US,25.05,41.3712,-73.4140,
34,Asau,RO,26.96,46.4333,26.4000,
36,Makat,KZ,22.26,47.6469,53.3463,
41,Tasiilaq,GL,22.69,65.6145,-37.6368,
46,Albany,US,18.19,42.6001,-73.9662,
65,Gornyak,RU,21.96,50.9942,81.4661,
72,Palmer,US,25.25,42.1584,-72.3287,
88,Whitehorse,CA,21.97,60.7161,-135.0538,
100,Yinchuan,CN,27.70,38.4681,106.2731,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)
output_data_file = ("../WeatherPyWeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
